This will plot and compare the resulting data. To run this, make sure that you have already generated all data you want to be compared. For instructions on how to run the generation, see README-new.md

In [11]:
import os
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import json

from cloudmesh.common.Shell import Shell
from cloudmesh.common.parameter import Parameter
from cloudmesh.common.dotdict import dotdict

from cloudmesh.common.util import banner
from analysis import get_data, read_log, generate_df
from analysis import generate_average

In [2]:
df = generate_df()

In [3]:
df

,p,size,repeat,sort,user,node,t,c,time
0,18,100,10,mp,alex,v100,None,None,0.088
1,18,100,10,mp,alex,v100,None,None,0.076
2,18,100,10,mp,alex,v100,None,None,0.082
3,18,100,10,mp,alex,v100,None,None,0.075
4,18,100,10,mp,alex,v100,None,None,0.077
...,...,...,...,...,...,...,...,...,...
475,23,50,10,mp,alex,v100,None,None,0.092
476,23,50,10,mp,alex,v100,None,None,0.073
477,23,50,10,mp,alex,v100,None,None,0.095
478,23,50,10,mp,alex,v100,None,None,0.097


In [5]:
print(df['time'].dtype)

float64


In [9]:
def average_df(df):
    _df = df.groupby(['p','size','repeat','sort','user','node','t','c'], sort=True, as_index=False).agg({'time':'mean'})
    _df.reset_index()
    return _df

In [10]:
_df = average_df(df)
_df

,p,size,repeat,sort,user,node,t,c,time
0,1,50,10,mp,alex,v100,None,None,0.0251
1,1,100,10,mp,alex,v100,None,None,0.0152
2,2,50,10,mp,alex,v100,None,None,0.0181
3,2,100,10,mp,alex,v100,None,None,0.0172
4,3,50,10,mp,alex,v100,None,None,0.0208
5,3,100,10,mp,alex,v100,None,None,0.0235
6,4,50,10,mp,alex,v100,None,None,0.0268
7,4,100,10,mp,alex,v100,None,None,0.0245
8,5,50,10,mp,alex,v100,None,None,0.0332
9,5,100,10,mp,alex,v100,None,None,0.0333


In [8]:
def get_values(df, processes, sizes, sorts, users, nodes):
    _df = df[df['p'].isin(processes) & df['size'].isin(sizes) & df['sort'].isin(sorts) & df['user'].isin(users) & df['node'].isin(nodes)]
    return _df

In [9]:
_df = get_values(df, [1, 2, 3], [50], ["mp"], ["alex"], ["v100"])

In [ ]:
def calculate_speedup(df):
    

In [10]:
_df

,p,size,repeat,sort,user,node,t,c,time
10,2,50,10,mp,alex,v100,None,None,0.028
11,2,50,10,mp,alex,v100,None,None,0.017
12,2,50,10,mp,alex,v100,None,None,0.017
13,2,50,10,mp,alex,v100,None,None,0.016
14,2,50,10,mp,alex,v100,None,None,0.018
15,2,50,10,mp,alex,v100,None,None,0.016
16,2,50,10,mp,alex,v100,None,None,0.017
17,2,50,10,mp,alex,v100,None,None,0.017
18,2,50,10,mp,alex,v100,None,None,0.017
19,2,50,10,mp,alex,v100,None,None,0.018


In [ ]:
def plot_benchmark_by_size(df, size=None, tag=None, files=None, x="sizes", y="time"):
    names = "-".join(files)
    sns.lineplot(data=df, x=x, y=y, hue="name");
    t = tag.title()
    label = f"{t}".replace("_", " ")
    plt.title(f"{label}")
    if "time" in y: y = f"{y}/s"
    if "time" in x: x = f"{x}/s"
    plt.xlabel(x.capitalize())
    plt.ylabel(y.capitalize())
    plt.savefig(f"images/{tag}-{names}.png")
    plt.savefig(f"images/{tag}-{names}.pdf")

In [ ]:
def plot()

In [ ]:
df

In [ ]:
df = df.pivot_table(
    values='time', index=['tag', 'processes'], columns=['name', 'size'], fill_value=0, aggfunc='mean')

In [ ]:
df

In [11]:
print(df.loc['mp-mergesort'])
speedup = df.rdiv(df.loc['seq-merge'].iloc[0])
speedup

KeyError: 'mp-mergesort'